In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
drake_sad = pd.read_csv('playlist_1.csv')
drake_hype = pd.read_csv('playlist_2.csv')
drake_chill = pd.read_csv('playlist_3.csv')
drake_romantic = pd.read_csv('playlist_4.csv')
drake_party = pd.read_csv('playlist_5.csv')

1. Split data into train and test
2. Run the recommendations on each song in the playlist, add recommendations to a recommendation list
3. Check if the recommendations are accurate with the test data

In [5]:
drake_sad

,track_uri,duration_ms,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,spotify:track:047fCsbO4NdmwCBn8pcUXl,347227,0.492,0.260,9,-17.341,0.0921,0.6460,0.001780,0.0705,0.3120,111.519
1,spotify:track:2Gnsof1hvZzjE1xdLRpjtf,272573,0.489,0.570,1,-10.291,0.3060,0.7590,0.000002,0.1790,0.5840,185.516
2,spotify:track:6Z01gUquJsjJC67uNWm6P0,224720,0.566,0.465,2,-11.714,0.2350,0.0555,0.000000,0.5670,0.1770,143.015
3,spotify:track:2KvHC9z14GSl4YpkNMX384,283551,0.618,0.693,7,-5.943,0.4500,0.2460,0.000000,0.1120,0.4540,170.982
4,spotify:track:4wVOKKEHUJxHCFFNUWDn0B,220488,0.735,0.449,10,-7.507,0.3470,0.6290,0.000000,0.1130,0.0397,122.947
5,spotify:track:7JXZq0JgG2zTrSOAgY8VMC,320400,0.697,0.235,7,-8.651,0.0534,0.6200,0.000144,0.1070,0.3780,99.947
6,spotify:track:0w1ZtnzQmtmuuoKxHT0pLL,247893,0.776,0.482,1,-11.268,0.0356,0.4810,0.045700,0.1100,0.1280,136.015
7,spotify:track:0GaBIpyHvytM1UBYmqXu08,322160,0.739,0.491,1,-9.726,0.4130,0.6260,0.000000,0.1260,0.4480,94.391
8,spotify:track:124NFj84ppZ5pAxTuVQYCQ,277387,0.629,0.515,0,-10.358,0.2650,0.0267,0.000012,0.0888,0.2990,121.845
9,spotify:track:6UgxOrbHWJeqHvh0ost7Aa,261853,0.681,0.569,6,-5.859,0.5460,0.1730,0.000000,0.1800,0.1470,140.532
